In [1]:
import json
tree_data = {
    "node1": {
        #"parent": None,
        "data": "Some data for node1",
        "more": "Some more data for node1"
    },
    "node2": {
        #"parent": "node1",
        "data": "Some data for node2"
    },
    "node3": {
        "parent": "node1",
        "data": "Some data for node3",
        "test": "Some test data for node3"
    },
    "node4": {
        "parent": "node3",
        "data": "Some data for node4"
    },
    "node5": {
        "parent": "node3",
        "data": "Some data for node5"
    }
}
print(json.dumps(tree_data, indent=2))

{
  "node1": {
    "data": "Some data for node1",
    "more": "Some more data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3",
    "test": "Some test data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  }
}


In [2]:
import treekit as tk

# load a tree from tree_data
tree = tk.FlatTree(tree_data)#, metadata={ "name": "My Tree" })
print(json.dumps(tree, indent=2))


{
  "node1": {
    "data": "Some data for node1",
    "more": "Some more data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3",
    "test": "Some test data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  }
}


In [3]:
tree.root.payload

{}

In [4]:
node = tree.root
def pretty_print(node, depth=0):
    if depth != 0:
        print("    "*depth, "|")
        print("    "*depth, "+", "-"*depth, end=" ")
    print(node.name)
    for child in node.children:
        pretty_print(child, depth+1)
pretty_print(node)

__ROOT__
     |
     + - node1
         |
         + -- node3
             |
             + --- node4
             |
             + --- node5
     |
     + - node2


For now on, when viewing trees, we use the `TreeViz` class to visualize the trees.
It has a `text` and `image` method to display the tree in text and image format respectively.

In [5]:
from treekit.tree_viz import TreeViz
from treekit.tree_converter import TreeConverter
anytree = TreeConverter.to_anytree(tree.get_node("node1"),
                                   node_name=lambda node: node.name,
                                   data_extractor=None)

In [6]:
print(TreeViz.text(anytree))

node1
└── node3
    ├── node4
    └── node5



In [7]:
print(TreeViz.text(anytree, node_name=lambda node: node.name))

node1
└── node3
    ├── node4
    └── node5



In [8]:
#TreeConverter.to_treenode(anytree)

In [9]:

print(anytree.children[0].depth)
print(anytree.children[0].path)
print(anytree.children[0].ancestors)
print(anytree.children[0].siblings)
print(anytree.is_root)

1
(Node('/node1'), Node('/node1/node3'))
(Node('/node1'),)
()
True


In [10]:
TreeViz.image(anytree, "anytree.png")

Here is the image (`tree.png` from above) of the tree:

![tree](./tree.png)

We may want to show different or more information in the image.
In order to do so, firs we shold convert `tree` to any `anytree.Node` that
carries more information.

In [11]:
def detailed_node_info(node):
    return { 'data': node.payload['data'] if 'data' in node.payload else "None" }

anytree2 = TreeConverter.to_anytree(tree,
                                   node_name=lambda node: node.name,
                                   data_extractor=detailed_node_info)


TreeViz.image(anytree2, "anytree-more.png", node_name=lambda node: node.name + ": " + node.data + "\nchildren: " + str(len(node.children)))

Here is the image:

![tree](./anytree-more.png)


The `FlatTree` class provides a **view** of a `dict` object as a tree. We do not modify
the `dict` passed into it (and you can create a dict through the `FlatTree` API).

The `FlatTree` class has a number of methods and properties to help you navigate the tree.
A particular aspect of the `FlatTree` class is that it unifies any `dict` object into a tree
structure. The keys are the node names and the values are the node values. If
the value has no parent, it is a child of a `LOGICAL_ROOT` node that is computed
lazily on demand (and is not a part of the actual underlying `dict` object).
In this way, every dict is a tree, and every tree is a dict, with the exception
that undefined behavior may result if the `dict` has keys that map to values
in which a `parent` key is defined but results in a cycle or a node that is not
in the `dict`. In this case, it will still try to work with it, but the behavior
is undefined. You can call `FlatTree.check_valid` to check if the tree is in a
valid state.

Since it's just a view of a `dict` we have all the normal operations on it that
we would have on a `dict` object.

We can also use the `FlatTree` class to visualize sub-trees rooted at some node.

By default, the `FlatTree` conceptually represents as a whole the logical root
of the tree. However, we have `FlatTree.ProxyNode` objects that can be used to
represent any node in the tree, and supports the same API as the `FlatTree` class
itself.

In [12]:
print(TreeViz.text(TreeConverter.to_anytree(tree.get_node("node3"))))

node3
├── node4
└── node5



In [13]:
print(tree["node2"])
node3 = tree.get_node("node3")
print(node3)
print(node3['parent'])
print(node3.children)

{'data': 'Some data for node2'}
ProxyNode({'parent': 'node1', 'data': 'Some data for node3', 'test': 'Some test data for node3'})
node1
[ProxyNode({'parent': 'node3', 'data': 'Some data for node4'}), ProxyNode({'parent': 'node3', 'data': 'Some data for node5'})]


We show that it's easy to regenerate any JSON files that may have been used
to generate the FlatTree 'tree'. So, JSON is a good format for storing and
transmitting trees. And, of course, `FlatTree` *is* a dictionary. Of course,
if we store an object that has no serializable representation, it cannot be
stored in JSON.

In [14]:
print(json.dumps(tree,  indent=2) == json.dumps(tree_data, indent=2))

True


In [15]:

# let's create a tree from a dict that cannot be serialized to json
non_serializable_tree_data = {
    "node1": {
        #"parent": None,
        # data is a function that cannot be serialized to json
        "data": lambda x: 2*x**3 + 3*x**2 + 4*x + 5
    }
}

non_serializable_tree = tk.FlatTree(non_serializable_tree_data)
print(non_serializable_tree)
print(non_serializable_tree.root)

try:
    json.dumps(non_serializable_tree, indent=2)
except TypeError as e:
    print(e)

{'node1': {'data': <function <lambda> at 0x7087d521a8e0>}}
ProxyNode({})
Object of type function is not JSON serializable


In [16]:
child = tree.root.add_child(key="node36", data="Some data for node36")
print(child)
tree.get_node("node1").add_child(key="node15", data="Some data for node15")
print(TreeViz.text(TreeConverter.to_anytree(tree)))

ProxyNode({'data': 'Some data for node36'})
__ROOT__
├── node1
│   ├── node3
│   │   ├── node4
│   │   └── node5
│   └── node15
├── node2
└── node36



If we try too add a non-unique node key to the tree, we will get a `KeyError`.

In [17]:
try:
    child2 = tree.get_node("node1").add_child(key="node2", data="Some data for node2!!!")
except KeyError as e:
    print(e)

'Node key already exists in the tree'


Let's add some more nodes.

In [18]:
child7 = child.add_child(key="node7", data="Some data for node7")
child7.add_child(key="node17", data="Some data for node17")
child7.add_child(key="node27", data="Some data for node27").add_child(key="node37", data="Some data for node37")
print(TreeViz.text(TreeConverter.to_anytree(child7), node_name=lambda n: n))

Node('/node7', data='Some data for node7')
├── Node('/node7/node17', data='Some data for node17')
└── Node('/node7/node27', data='Some data for node27')
    └── Node('/node7/node27/node37', data='Some data for node37')



Let's view the full tree with the `TreeViz.image` method.

In [19]:
TreeViz.image(TreeConverter.to_anytree(tree), "big_tree.png")

Here's the tree:

![big_tree](./big_tree.png)

We can see that the underlying tree that the child is a member of has been
appropriately modified:

We can iterate over the items of the child and we can modify/delete its data.

In [20]:
for k, v in child7.items():
    print(k)

child7["new_data"] = "Some new data for node7"    
print(child7)

del child7["new_data"]
print(child7)

child7["other_new_data"] = "Some other data for node7"


data
parent
ProxyNode({'data': 'Some data for node7', 'parent': 'node36', 'new_data': 'Some new data for node7'})
ProxyNode({'data': 'Some data for node7', 'parent': 'node36'})


Let's create a tree from a dictionary that refers to a non-existent parent.

In [21]:
try:
    non_existent_parent_tree = tk.FlatTree({
        "node1": {
            "parent": "non_existent_parent",
            "data": "Some data for node1"
        }})
    print(TreeViz.text(TreeConverter.to_anytree(non_existent_parent_tree)))
    non_existent_parent_tree.check_valid()
except KeyError as e:
    print(e)



__ROOT__

"Parent node non-existent: 'non_existent_parent'"


We see that the node is disconnected from the logical root, since it refers
to a non-existent parent.

In [22]:

try:
    cycle_tree = tk.FlatTree({
        "node0": {
            "parent": None,
            "data": "Some data for node0"
        },
        "node1": {
            "parent": "node2",
            "data": "Some data for node1"
        },
        "node2": {
            "parent": "node3",
            "data": "Some data for node2"
        },
        "node3": {
            "parent": "node1",
            "data": "Some data for node3"
        },
        "node4": {
            "parent": "node0",
            "data": "Some data for node4"
        }
    })

    print(TreeViz.text(TreeConverter.to_anytree(cycle_tree)))
    cycle_tree.check_valid()
except ValueError as e:
    print(e)

Cycle detected: {'node2', 'node3', 'node1'}


We see that the tree was in an invalid state. In particular, nodes 1, 2, and 3
are disconnected from the logical root and in a cycle. We can fix this by
breaking the cycle and setting the parent of node 3 to, for instance, the
logical root (by setting it to `None`).

In [23]:
cycle_tree['node3']['parent'] = None
print(TreeViz.text(TreeConverter.to_anytree(cycle_tree)))

__ROOT__
├── node0
│   └── node4
└── node3
    └── node2
        └── node1



If you modify the tree structure using the `FlatTree` API, not the `dict` API,
we actually do enforce the tree structure. For example, if we try to set the
parent of a node to a non-existent node, we will get a `KeyError`.

In [24]:
from copy import deepcopy
try:
    deepcopy(tree).get_node("node2")['parent'] = 'node14994'
except KeyError as e:
    print(e)


"Parent node non-existent: 'node14994'"


Let's look at the tree again, and see about creating a cycle.

In [25]:
print(TreeViz.text(TreeConverter.to_anytree(tree)))

__ROOT__
├── node1
│   ├── node3
│   │   ├── node4
│   │   └── node5
│   └── node15
├── node2
└── node36
    └── node7
        ├── node17
        └── node27
            └── node37



We will make node 1 the parent of node 5, to create a cycle:

In [26]:
try:
    deepcopy(tree).get_node("node1")['parent'] = 'node5'
except ValueError as e:
    print(e)

Cycle detected: {'node5', 'node3', 'node1'}


Notice that we use `deepcopy` to avoid modifying the original tree with these
invalid operations. We chose to do it this way so as to not incur the overhead
of reverting the tree to a valid state after an invalid operation. This way,
we can keep the tree in an invalid state for as long as we want, and only
revert it to a valid state when we want to.

Each node is a key-value pair in the `FlatTree`. We have the  `FlatTree.ProxyNode`
so that we can have an API focused on the nodes and not the underlying dictionary.
However, we stiill permit access to the underlying dictionary. When you modify
the tree in this way, we still maintain the integrity of the tree.

Since the `FlatTree` represents nodes as key-value pairs, and the value may
have a parent key, along with any other arbitrary data, each value for a node
must be a dictionary.

Below, we see that trying to add a `test` node with a non-dictionary value
generates an error.

In [27]:
try:
    error_tree = deepcopy(tree)
    error_tree['test'] = "Some test data"
    error_tree.check_valid()
except ValueError as e:
    print(e)


Node test's value must be a dictionary: value='Some test data'


Let's manipulate the tree a bit more using the `dict` API. We're just going to
add a `new_node` with some data.

In [28]:

tree['new_node'] = {'parent': 'node1', 'data': 'Some new data'}
print(TreeViz.text(TreeConverter.to_anytree(tree)))

__ROOT__
├── node1
│   ├── node3
│   │   ├── node4
│   │   └── node5
│   ├── node15
│   └── new_node
├── node2
└── node36
    └── node7
        ├── node17
        └── node27
            └── node37



In [29]:
print(json.dumps(tree, indent=2))


{
  "node1": {
    "data": "Some data for node1",
    "more": "Some more data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3",
    "test": "Some test data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node36"
  },
  "node15": {
    "data": "Some data for node15",
    "parent": "node1"
  },
  "node7": {
    "data": "Some data for node7",
    "parent": "node36",
    "other_new_data": "Some other data for node7"
  },
  "node17": {
    "data": "Some data for node17",
    "parent": "node7"
  },
  "node27": {
    "data": "Some data for node27",
    "parent": "node7"
  },
  "node37": {
    "data": "Some data for node37",
    "parent": "node27"
  },
  "new_node": {
    "parent": "node1",
    "data": "Some new data"
  }
}


The logical root node is not a part of the underlying dictionary, so we can't
access it through the `dict` API. It's non-children data are also immutable
through the `FlatTree` API.

In [30]:
root_node = tree.root
print(root_node)

try:
    root_node['data'] = "Some new data for root node"
except TypeError as e:
    print(e)

try:
    root_node['parent'] = None
except TypeError as e:
    print(e)

try:
    root_node.clear()
except TypeError as e:
    print(e)


ProxyNode({})
ProxyNode({}) is immutable
ProxyNode({}) is immutable


We can *detach* nodes. Let's first view the full tree, pre-detachment.

In [31]:
TreeViz.image(TreeConverter.to_anytree(tree), "full-tree-pre-detach-or-prune.png")

Here's the full tree prior to any detachment or pruning:

![full-tree-pre-detach-or-prune](./full-tree-pre-detach-or-prune.png)

Let's detach node 36 and see the tree.

In [33]:
tree.get_node("node36").detach()
print(json.dumps(tree, indent=2))

{
  "node1": {
    "data": "Some data for node1",
    "more": "Some more data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3",
    "test": "Some test data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node36",
    "parent": "__DETACHED__"
  },
  "node15": {
    "data": "Some data for node15",
    "parent": "node1"
  },
  "node7": {
    "data": "Some data for node7",
    "parent": "node36",
    "other_new_data": "Some other data for node7"
  },
  "node17": {
    "data": "Some data for node17",
    "parent": "node7"
  },
  "node27": {
    "data": "Some data for node27",
    "parent": "node7"
  },
  "node37": {
    "data": "Some data for node37",
    "parent": "node27"
  },
  "new_node": {
    "parent": "node1",
    "data": "Some new data"
  

In [ ]:
detached_node_36_tree = TreeConverter.to_anytree(tree)
TreeViz.image(detached_node_36_tree, "full-tree-post-detach-node-36.png")

Let's view the tree with the detached node 36.

![full-tree-post-detach-node-36](./full-tree-post-detach-node-36.png)

We see that node 36 (and any of its children) are no longer attached to the logical root, which represents the root of the tree.
However, it's not gone -- it's detached. We can view it with:

In [ ]:
node36 = tree.get_node("node36")
TreeViz.image(TreeConverter.to_anytree(node36), "node-36-pre-prune.png")

Here's the tree prior to the pruning.

![node-36-pre-prune.png](./node-36-pre-prune.png)

We can *prune* any detached nodes. Let's prune the detached nodes.

In [ ]:
tree.prune_detached()
try:
    tree.get_node("node36")
except KeyError as e:
    print(e)


"Node not found: 'node36'"


In [ ]:
try:
    tree.get_node("node17")
except KeyError as e:
    print(e)

"Node not found: 'node17'"


So, after detaching node 36 and then pruning detached nodes, we only have the
nodes in the attached tree. It'll still show the same as before, but we can
look at the underlying `dict` to verify the detached nodes are gone.

In [ ]:
print(json.dumps(tree, indent=2))

{
  "node1": {
    "data": "Some data for node1",
    "more": "Some more data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3",
    "test": "Some test data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node15": {
    "data": "Some data for node15",
    "parent": "node1"
  },
  "new_node": {
    "parent": "node1",
    "data": "Some new data"
  }
}


We can also directly prune nodes from the tree. Let's prune node 3.

In [ ]:
tree.prune("node3")
try:
    tree.get_node("node3")
except KeyError as e:
    print(e)

TreeViz.image(TreeConverter.to_anytree(tree), "post-prune-node-3.png")

"Node not found: 'node3'"



Let's view the tree after pruning node 3.


![post-prune-node-3](./post-prune-node-3.png)

We have a fairly complete API for manipulating the tree. Let's explore some
additional methods.

In [ ]:
tree.get_node("node1").clear()
#print(tree.get_node("node1"))
tree.get_node("node1")['new_data'] = "Some new data for node1"
tree.get_node("node1")['other_new_data'] = "Some other data for node1"
tree.get_node("node1").add_child(key="node77", data="Some data for node77")

print(TreeViz.text(TreeConverter.to_anytree(tree, node_name=lambda n: n.payload)))

{}
├── {'new_data': 'Some new data for node1', 'other_new_data': 'Some other data for node1'}
│   ├── {'data': 'Some data for node15', 'parent': 'node1'}
│   ├── {'parent': 'node1', 'data': 'Some new data'}
│   └── {'data': 'Some data for node77', 'parent': 'node1'}
└── {'data': 'Some data for node2'}



This is fairly self-expalanatory. Let's add some more nodes without specifying
a key name for them, since often we don't care about the key name and it's
only for bookkeeping purposes.

In [ ]:
tree.add_child(whatever=3).add_child(whatever=4).add_child(whatever=5)
print(TreeViz.text(TreeConverter.to_anytree(tree)))

__ROOT__
├── node1
│   ├── node15
│   ├── new_node
│   └── node77
├── node2
└── 43f24f40-fd3c-446b-af98-ba79bfa9b228
    └── 8b35ade0-f744-4f43-b634-cca76cfa0f8d
        └── bf367759-37d0-4595-bbe5-7c2b25ca6156



Let's look at some tree conversions. We can convert between different tree
representations and data structures.

In [ ]:
import treekit.tree_converter as tc
new_tree = tc.TreeConverter.to_treenode(deepcopy(tree), node_name = lambda node: node.name, data_extractor = lambda node: node.payload)
print(type(new_tree))

<class 'treekit.treenode.TreeNode'>


We see that it's a different type of tree, a `TreeNode`, which is a recursive
data structure. Nowever, it's also a `dict`. Let's look at its representation.

In [ ]:
print(json.dumps(new_tree, indent=2))

{
  "__name__": "__ROOT__",
  "children": [
    {
      "__name__": "node1",
      "new_data": "Some new data for node1",
      "other_new_data": "Some other data for node1",
      "children": [
        {
          "__name__": "node15",
          "data": "Some data for node15",
          "parent": "node1",
          "children": []
        },
        {
          "__name__": "new_node",
          "parent": "node1",
          "data": "Some new data",
          "children": []
        },
        {
          "__name__": "node77",
          "data": "Some data for node77",
          "parent": "node1",
          "children": []
        }
      ]
    },
    {
      "__name__": "node2",
      "data": "Some data for node2",
      "children": []
    },
    {
      "__name__": "43f24f40-fd3c-446b-af98-ba79bfa9b228",
      "whatever": 3,
      "children": [
        {
          "__name__": "8b35ade0-f744-4f43-b634-cca76cfa0f8d",
          "whatever": 4,
          "parent": "43f24f40-fd3c-446b-af98-ba79

We see that it has a very different structure. However, when we pretty-print
it using `TreeViz`, we see that it's the same tree.

In [ ]:
print(TreeViz.text(TreeConverter.to_anytree(new_tree)))

__ROOT__
├── node1
│   ├── node15
│   ├── new_node
│   └── node77
├── node2
└── 43f24f40-fd3c-446b-af98-ba79bfa9b228
    └── 8b35ade0-f744-4f43-b634-cca76cfa0f8d
        └── bf367759-37d0-4595-bbe5-7c2b25ca6156



We also support conversions to and from `FlatTree`, `TreeNode`, and `anytree.Node` objects.

The `TreeNode` is a bit more useful for operations that require recursion, but
any tree can support the sae operations. The `TreeNode` is a bit more specialized
for this purpose, and the `FlatTree` is a bit more specialized for more general
storage and manipulation of data that is tree-like, such as configuration data
or log data. See `TreeNode.md` for more information on the `TreeNode` class.